# NLP

In [2]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
%pip install pyspark==3.4.0

# install spark-nlp
%pip install spark-nlp==5.1.3

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.3.0



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - openjdk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.3.11  |       h06a4308_0         127 KB
    certifi-2024.2.2           |  py310h06a4308_0         159 KB
    openjdk-11.0.13            |       h87a67e3_0       341.0 MB
    ------------------------------------------------------------
                                           Total:       341.3 MB

The following NEW packages will be INSTALLED:

  openjdk            pkgs/main/li

In [3]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    .getOrCreate()
)

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4ec160d1-d6c1-43a6-87de-479640bbcb4a;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 442ms :: artifacts dl 51ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

3.4.0


In [4]:
import sagemaker
session = sagemaker.Session()
bucket = session.default_bucket()
output_prefix_data_submissions = "project/submissions/yyyy=*"
s3_path = f"s3a://{bucket}/{output_prefix_data_submissions}"
print(f"reading submissions from {s3_path}")
posts = spark.read.parquet(s3_path, header=True)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
reading submissions from s3a://sagemaker-us-east-1-562166416351/project/submissions/yyyy=*


24/04/12 23:22:21 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/04/12 23:22:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
#reading comments
output_prefix_data_comments = "project/comments/yyyy=*"
s3_path = f"s3a://{bucket}/{output_prefix_data_comments}"
comments = spark.read.parquet(s3_path, header=True)

In [6]:
import pyspark.sql.functions as f
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

# Adding variables

In [7]:
comments = comments.withColumn('misinfo_class', 
                    f.when(comments.body.rlike(r'fake news|bullshit|misinfo|clickbait|unreliable|propoganda'), True)\
                    .otherwise(False))

# LDA

In [8]:
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml import Pipeline
from nltk.corpus import stopwords
from pyspark.ml.feature import CountVectorizer , IDF
from pyspark.ml.clustering import LDA
import pyspark.sql.functions as f
from pyspark.sql.types import StringType, ArrayType, FloatType
from itertools import chain

In [80]:
#create small df to use for LDA
small_df = posts.select('title', 'id')

In [81]:
#create tokenizer
tokenizer = Tokenizer(outputCol="words")
tokenizer.setInputCol("title")

Tokenizer_3001da9cc20b

In [91]:
#remove stop words 
StopWords = stopwords.words("english")
#removing stop words in other languages and other common words
additional = ['@reuters:', '–' '&amp;', '@ap:', 'rt', ':', 'از', 'آهنگ', 'دانلود', 'در', 'به', 'جدید', '@apentertainment:',
             '|', 'के', 'में', 'و', 'في', 'من', '@bbcworld:', 'de', 'la', 'di', 'की', 'से', 'bio', 'many','know', 'age', 'says', 'one',
             'net', 'user]', '[deleted', 'look', '–']
StopWords = StopWords + additional
remover = StopWordsRemover(stopWords=StopWords)
remover.setInputCol("words")
remover.setOutputCol("filtered")

StopWordsRemover_78f648a7701f

In [92]:
#count vectorizer
cv = CountVectorizer(inputCol="filtered", outputCol="raw_features", vocabSize=5000, minDF=25)
# IDF
idf = IDF(inputCol="raw_features", outputCol="features")

In [93]:
#lda model 
lda = LDA()
lda.setK(8)
lda.setMaxIter(10)
lda.setSeed(13)

LDA_049120290467

In [94]:
pipeline = Pipeline(stages=[tokenizer, remover, cv, idf, lda])

In [95]:
model = pipeline.fit(small_df)

In [96]:
topics = model.stages[-1].describeTopics()
terms = model.stages[-3].vocabulary

In [97]:
#get word from index of term 
def indices_to_terms(indices, terms=terms):
        terms_subset = [terms[index] for index in indices]
        return terms_subset
# Defining Spark UDF from above function
udf_indices_to_terms = f.udf(indices_to_terms, ArrayType(StringType()))

topics = (
    topics
       .withColumn("terms", udf_indices_to_terms(f.col("termIndices")))
    )

In [98]:
topics.take(20)

[Row(topic=0, termIndices=[34, 3, 1, 77, 58, 20, 178, 5, 180, 171], termWeights=[0.00904600934488922, 0.008137299161650556, 0.007184898341812056, 0.006787143963824302, 0.0059616974769255825, 0.0055684418792829775, 0.005381365341505666, 0.005261148762084106, 0.005117269724229448, 0.004803771663356002], terms=['age,', 'ukraine', 'new', 'height,', 'indian', '&amp;', 'crypto', 'russian', 'worth,', 'wiki,']),
 Row(topic=1, termIndices=[64, 166, 2, 72, 14, 298, 8, 13, 1, 29], termWeights=[0.007901737969180506, 0.005648034526313718, 0.005513242986136079, 0.004696365473711784, 0.004647664488141271, 0.004256990903518756, 0.003943864923942797, 0.003725208719034877, 0.0035226697705458415, 0.0031740871562966613], terms=['free', 'crack', 'u.s.', 'key', 'biden', 'download', 'first', 'president', 'new', 'say']),
 Row(topic=2, termIndices=[0, 26, 8, 2, 1, 7, 12, 217, 269, 95], termWeights=[0.005559618241758851, 0.004768308519960471, 0.003980994305342993, 0.003495670114078333, 0.0034946457273576583, 0.

In [99]:
#naming topics 
topic_dict = {0: 'economics/russia&ukraine', 1: 'presidental news', 2: 'supreme court/law', 3: 'global politics', 4: 'us politics', 
              5: 'covid/russia&ukraine', 6: 'crime/protest', 7: 'tv shows'}
              

In [100]:
small_transform = model.transform(small_df)

In [101]:
#map to topics
mapping_expr = f.create_map([f.lit(x) for x in chain(*topic_dict.items())])

In [102]:
#udf to get the top topic 
max_topic = f.udf(lambda v:float(v.argmax()),FloatType())
#using mao and udf to create a topic column
topic = small_transform.withColumn('topic_num', max_topic("topicDistribution"))\
.withColumn("topic", mapping_expr[f.col("topic_num")]).select('id','topic')

In [103]:
mini_posts = posts.select('created_utc', 'title', 'id')

In [105]:
#merging relevant columns wuth the topic column
merged_df = mini_posts.join(topic, 'id')

In [112]:
merged_df.take(5)

[Row(id='10000r8', created_utc=datetime.datetime(2022, 12, 31, 18, 10, 56), title='Who dares bins? Councils in England use ex-SAS soldiers to catch fly-tippers', topic='us politics'),
 Row(id='10004rz', created_utc=datetime.datetime(2022, 12, 31, 18, 16, 6), title='Iran tests military drones in wargame near Strait of Hormuz', topic='covid/russia&ukraine'),
 Row(id='1000b7b', created_utc=datetime.datetime(2022, 12, 31, 18, 24, 30), title='Wise Registry Cleaner Pro 11.3.4 Crack Here [2023]', topic='presidental news'),
 Row(id='1000e7x', created_utc=datetime.datetime(2022, 12, 31, 18, 28, 22), title='ai-Marketing', topic='economics/russia&ukraine'),
 Row(id='1000o6u', created_utc=datetime.datetime(2022, 12, 31, 18, 41, 24), title='VuzeVPN Not Responding Big Sur 1.0.8.1 Crack Activation Key', topic='presidental news')]

## Adding comments

In [129]:
#renaming columns and removing the t3_ from the link id to get the post id on the comment
mini_comments = comments.select('created_utc','body','misinfo_class', 'link_id', 'id')\
.withColumn('comment_created', f.col('created_utc')).withColumn('comment_id', f.col('id'))\
.withColumn('id', f.regexp_extract('link_id', 't3_(.*)$', 1))

In [132]:
#merging dataframes
total_df = merged_df.join(mini_comments, 'id')

## Counting by topic

In [133]:
topic_misinfo_counts = total_df.filter(f.col('misinfo_class') == True).groupBy('topic').count().toPandas()

In [134]:
topic_misinfo_counts

,topic,count
0,us politics,24261
1,economics/russia&ukraine,47848
2,crime/protest,90683
3,global politics,35838
4,presidental news,42871
5,tv shows,14998
6,covid/russia&ukraine,91546
7,supreme court/law,59082


In [135]:
topic_misinfo_counts.to_csv('../data/csv/topic_misinfo_true_count.csv', index = False)

In [136]:
topic_counts = total_df.groupBy('topic').count().toPandas()

24/04/13 04:56:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/13 04:56:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/13 04:56:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/13 04:56:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/13 04:56:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/13 04:56:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/13 04:56:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/13 04:56:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/13 04:56:36 WARN RowBasedKeyValueBatch: Calling spill() on

In [137]:
topic_counts

,topic,count
0,us politics,2761048
1,economics/russia&ukraine,5721107
2,crime/protest,11658072
3,global politics,4542845
4,presidental news,5028581
5,tv shows,2008234
6,covid/russia&ukraine,11446475
7,supreme court/law,6738106


In [138]:
topic_counts.to_csv('../data/csv/topic_counts.csv', index = False)

In [139]:
topic_misinfo_total = total_df.groupBy(['topic', 'misinfo_class']).count().toPandas()

24/04/13 05:19:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/13 05:19:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/13 05:19:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/13 05:19:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/13 05:19:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/13 05:19:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/13 05:19:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/13 05:19:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/04/13 05:19:04 WARN RowBasedKeyValueBatch: Calling spill() on

In [140]:
topic_misinfo_total

,topic,misinfo_class,count
0,economics/russia&ukraine,True,47848
1,covid/russia&ukraine,False,11354929
2,supreme court/law,True,59082
3,global politics,True,35838
4,crime/protest,False,11567389
5,economics/russia&ukraine,False,5673259
6,supreme court/law,False,6679024
7,us politics,True,24261
8,crime/protest,True,90683
9,tv shows,True,14998


In [141]:
topic_misinfo_total.to_csv('../data/csv/topic_misinfo_total.csv', index = False)